In [1]:
#Importing required packages.
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
#from sklearn.linear_model import SGDClassifier
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
%matplotlib inline

In [2]:
rental_data = pd.read_csv("car_rental_huge.csv",low_memory=False)
rental_data.head(10)

,Unnamed: 0,Booking Number,Price,Origin Lat,Origin Lon,Dest Lat,Dest Lon,Wait,Distance,Transport,Day,Date,Month,Year,Hour,Haul,Time Taken
0,0,BA34665,1075.71,28.429399,-81.308998,40.639801,-73.778900,450,2500,0,5,17,7,2020,1,0,480
1,1,VA47151,2482.45,53.353699,-2.274950,53.630402,9.988230,0,1300,0,4,15,7,2021,1,0,17
2,2,FA59772,1390.11,53.353699,-2.274950,53.353699,-2.274950,60,1000,1,1,15,6,2020,1,0,63
3,3,BA88264,2315.39,45.470600,-73.740799,51.470600,-0.461941,90,800,1,4,18,3,2021,1,1,92
4,4,RY79338,2243.47,53.353699,-2.274950,51.470600,-0.461941,240,800,0,7,30,6,2019,1,0,250
5,5,VA83667,2004.28,51.470600,-0.461941,-33.946098,151.177002,210,100,0,1,13,1,2020,1,2,212
6,6,DL17990,931.63,28.429399,-81.308998,51.470600,-0.461941,240,800,1,7,10,2,2019,1,2,242
7,7,AA64640,1295.60,49.193901,-123.183998,40.639801,-73.778900,180,2500,0,3,7,8,2019,1,1,216
8,8,FA52560,467.46,53.353699,-2.274950,53.353699,-2.274950,60,1000,0,4,28,3,2019,1,0,73
9,9,RY18161,1329.35,-33.946098,151.177002,53.353699,-2.274950,180,1000,0,5,5,2,2021,1,2,196


In [3]:
rental_data['Origin Lat'].value_counts()

 28.429399    125316
 40.639801    125296
 51.470600    125156
 45.470600    124964
 53.630402    124865
-33.946098    124865
 49.193901    124821
 53.353699    124717
Name: Origin Lat, dtype: int64

In [4]:
rental_data['Dest Lat'].value_counts()

 53.630402    125337
 28.429399    125300
 40.639801    125132
 51.470600    124957
 53.353699    124906
 45.470600    124899
-33.946098    124792
 49.193901    124677
Name: Dest Lat, dtype: int64

In [5]:
from math import isclose
def getID(x):
    if(isclose(x, 53.630402, abs_tol=0.001)):
        return 0
    if(isclose(x, 49.193901, abs_tol=0.001)):
        return 1
    if(isclose(x, -33.946098, abs_tol=0.001)):
        return 2
    if(isclose(x, 45.470600, abs_tol=0.001)):
        return 3
    if(isclose(x, 40.639801, abs_tol=0.001)):
        return 4
    if(isclose(x, 28.429399, abs_tol=0.001)):
        return 5
    if(isclose(x, 53.353699, abs_tol=0.001)):
        return 6
    if(isclose(x, 51.470600, abs_tol=0.001)):
        return 7
    return -1
rental_data['OriginID'] = rental_data['Origin Lat'].apply(getID)
rental_data['DestID'] = rental_data['Dest Lat'].apply(getID)

In [6]:
from math import sin, cos, sqrt, atan2, radians
for index, row in rental_data.iterrows():
    
    
    
    R = 6373.0

    lat1 = radians(row['Origin Lat'])
    lon1 = radians(row['Origin Lon'])
    lat2 = radians(row['Dest Lat'])
    lon2 = radians(row['Dest Lon'])

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    rental_data.at[index,'Flight Distance'] = distance

In [7]:
rental_data.head()

,Unnamed: 0,Booking Number,Price,Origin Lat,Origin Lon,Dest Lat,Dest Lon,Wait,Distance,Transport,Day,Date,Month,Year,Hour,Haul,Time Taken,OriginID,DestID,Flight Distance
0,0,BA34665,1075.71,28.429399,-81.308998,40.639801,-73.778900,450,2500,0,5,17,7,2020,1,0,480,5,4,1521.866215
1,1,VA47151,2482.45,53.353699,-2.274950,53.630402,9.988230,0,1300,0,4,15,7,2021,1,0,17,6,0,811.086195
2,2,FA59772,1390.11,53.353699,-2.274950,53.353699,-2.274950,60,1000,1,1,15,6,2020,1,0,63,6,6,0.000000
3,3,BA88264,2315.39,45.470600,-73.740799,51.470600,-0.461941,90,800,1,4,18,3,2021,1,1,92,3,7,5215.839430
4,4,RY79338,2243.47,53.353699,-2.274950,51.470600,-0.461941,240,800,0,7,30,6,2019,1,0,250,6,7,242.887118


In [8]:
rental_data.head(10)

,Unnamed: 0,Booking Number,Price,Origin Lat,Origin Lon,Dest Lat,Dest Lon,Wait,Distance,Transport,Day,Date,Month,Year,Hour,Haul,Time Taken,OriginID,DestID,Flight Distance
0,0,BA34665,1075.71,28.429399,-81.308998,40.639801,-73.778900,450,2500,0,5,17,7,2020,1,0,480,5,4,1521.866215
1,1,VA47151,2482.45,53.353699,-2.274950,53.630402,9.988230,0,1300,0,4,15,7,2021,1,0,17,6,0,811.086195
2,2,FA59772,1390.11,53.353699,-2.274950,53.353699,-2.274950,60,1000,1,1,15,6,2020,1,0,63,6,6,0.000000
3,3,BA88264,2315.39,45.470600,-73.740799,51.470600,-0.461941,90,800,1,4,18,3,2021,1,1,92,3,7,5215.839430
4,4,RY79338,2243.47,53.353699,-2.274950,51.470600,-0.461941,240,800,0,7,30,6,2019,1,0,250,6,7,242.887118
5,5,VA83667,2004.28,51.470600,-0.461941,-33.946098,151.177002,210,100,0,1,13,1,2020,1,2,212,7,2,17025.997057
6,6,DL17990,931.63,28.429399,-81.308998,51.470600,-0.461941,240,800,1,7,10,2,2019,1,2,242,5,7,6967.532950
7,7,AA64640,1295.60,49.193901,-123.183998,40.639801,-73.778900,180,2500,0,3,7,8,2019,1,1,216,1,4,3931.621290
8,8,FA52560,467.46,53.353699,-2.274950,53.353699,-2.274950,60,1000,0,4,28,3,2019,1,0,73,6,6,0.000000
9,9,RY18161,1329.35,-33.946098,151.177002,53.353699,-2.274950,180,1000,0,5,5,2,2021,1,2,196,2,6,17017.658942


In [9]:
#X = rental_data[['DestID','Day','Date','Month','Distance','Transport']]
#X = rental_data[['DestID','Day','Transport','Haul']]
X = rental_data[['DestID','Day','Transport','Flight Distance']]
#X = rental_data[['Distance','Wait']]
y = rental_data['Time Taken']

In [10]:
X.head()

,DestID,Day,Transport,Flight Distance
0,4,5,0,1521.866215
1,0,4,0,811.086195
2,6,1,1,0.000000
3,7,4,1,5215.839430
4,7,7,0,242.887118


In [11]:
y.head()

0    480
1     17
2     63
3     92
4    250
Name: Time Taken, dtype: int64

In [12]:
#X_train = X.sample(frac=0.75)
#X_test = X.drop(X_train.index)
#y_train = y.sample(frac=0.75)
#y_test = y.drop(y_train .index)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [13]:
print(y)

0         480
1          17
2          63
3          92
4         250
         ... 
999995    368
999996     92
999997     63
999998    161
999999    100
Name: Time Taken, Length: 1000000, dtype: int64


In [14]:
#sc = StandardScaler()
#X_train = sc.fit_transform(X_train)
#X_test = sc.fit_transform(X_test)

In [15]:
rfc = RandomForestClassifier(n_estimators=10)
rfc.fit(X_train, y_train)
pred_rfc = rfc.predict(X_test)

In [16]:
print(classification_report(y_test, pred_rfc))

C:\Users\Nick\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           4       1.00      1.00      1.00      7132
          16       0.60      1.00      0.75      1583
          17       0.00      0.00      0.00      1252
          18       0.59      1.00      0.75      1079
          19       0.00      0.00      0.00       549
          20       0.51      0.59      0.55      1342
          21       0.49      0.40      0.44      1296
          34       1.00      1.00      1.00      5306
          38       1.00      1.00      1.00      8856
          46       0.59      1.00      0.74      1229
          47       0.00      0.00      0.00       985
          48       0.63      1.00      0.77       874
          49       0.00      0.00      0.00       383
          50       0.51      0.45      0.48       992
          51       0.54      0.60      0.57      1066
          60       0.00      0.00      0.00       678
          61       0.29      0.25      0.27      1017
          62       0.29    

In [17]:
print(confusion_matrix(y_test, pred_rfc))

[[7132    0    0 ...    0    0    0]
 [   0 1583    0 ...    0    0    0]
 [   0 1063    0 ...    0    0    0]
 ...
 [   0    0    0 ...    0    0    0]
 [   0    0    0 ...    0  865    0]
 [   0    0    0 ...    0    0 6222]]


In [18]:
from sklearn.metrics import accuracy_score
cm = accuracy_score(y_test, pred_rfc)

cm

0.784145

In [19]:
X.head()

,DestID,Day,Transport,Flight Distance
0,4,5,0,1521.866215
1,0,4,0,811.086195
2,6,1,1,0.000000
3,7,4,1,5215.839430
4,7,7,0,242.887118


In [20]:
y.head()

0    480
1     17
2     63
3     92
4    250
Name: Time Taken, dtype: int64

In [21]:
#Xnew = [[7,5,1,5,800,0]]
#ynew = rfc.predict(Xnew)
#print(ynew)

In [22]:
X_test.head()

,DestID,Day,Transport,Flight Distance
987231,5,7,0,15042.995529
79954,7,2,0,17025.997057
567130,2,3,0,0.000000
500891,0,4,1,7770.323440
55399,5,1,0,2009.023847


In [23]:
y_test.head()

987231    308
79954     103
567130    211
500891      4
55399     126
Name: Time Taken, dtype: int64

In [24]:
"""
temp_DestID = list(X_test['DestID'])
temp_Day = list(X_test['Day'])
temp_Date = list(X_test['Date'])
temp_Month = list(X_test['Month'])
temp_Distance = list(X_test['Distance'])
temp_Transport = list(X_test['Transport'])

temp_y = list(y_test)
differences = 0
offby=[0,0,0,0,0,0]

for i in range(len(y_test)):
    temp = rfc.predict([[temp_DestID[i],temp_Day[i],temp_Date[i],temp_Month[i],temp_Distance[i],temp_Transport[i]]])
    off = abs((temp-temp_y[i]))
    differences += off
    if(off==0):
        offby[0]+=1
    elif(off==1):
        offby[1]+=1
    elif(off==2):
        offby[2]+=1
    elif(off<=5):
        offby[3]+=1
    elif(off<=10):
        offby[4]+=1
    else:
        offby[5]+=1

#for i in range(len(y_test)):
"""

"\ntemp_DestID = list(X_test['DestID'])\ntemp_Day = list(X_test['Day'])\ntemp_Date = list(X_test['Date'])\ntemp_Month = list(X_test['Month'])\ntemp_Distance = list(X_test['Distance'])\ntemp_Transport = list(X_test['Transport'])\n\ntemp_y = list(y_test)\ndifferences = 0\noffby=[0,0,0,0,0,0]\n\nfor i in range(len(y_test)):\n    temp = rfc.predict([[temp_DestID[i],temp_Day[i],temp_Date[i],temp_Month[i],temp_Distance[i],temp_Transport[i]]])\n    off = abs((temp-temp_y[i]))\n    differences += off\n    if(off==0):\n        offby[0]+=1\n    elif(off==1):\n        offby[1]+=1\n    elif(off==2):\n        offby[2]+=1\n    elif(off<=5):\n        offby[3]+=1\n    elif(off<=10):\n        offby[4]+=1\n    else:\n        offby[5]+=1\n\n#for i in range(len(y_test)):\n"

In [25]:
temp_DestID = list(X_test['DestID'])
temp_Day = list(X_test['Day'])
temp_Transport = list(X_test['Transport'])
temp_Flight_Distance = list(X_test['Flight Distance'])

temp_y = list(y_test)
differences = 0
offby=[0,0,0,0,0,0]

for i in range(30003):
    temp = rfc.predict([[temp_DestID[i],temp_Day[i],temp_Transport[i],temp_Flight_Distance[i]]])
    off = abs((temp-temp_y[i]))
    differences += off
    if(off==0):
        offby[0]+=1
    elif(off==1):
        offby[1]+=1
    elif(off==2):
        offby[2]+=1
    elif(off<=5):
        offby[3]+=1
    elif(off<=10):
        offby[4]+=1
    else:
        offby[5]+=1

In [26]:
print(differences/30003)

[0.28890444]


In [27]:
print(offby)

[23596, 4764, 1130, 513, 0, 0]


In [28]:
count = 0
for i in offby:
    print("Off by %d: %f%%"% (count, i/30003*100))
    count+=1

Off by 0: 78.645469%
Off by 1: 15.878412%
Off by 2: 3.766290%
Off by 3: 1.709829%
Off by 4: 0.000000%
Off by 5: 0.000000%


In [29]:
for i in range(1,10,1):
    rfc = RandomForestClassifier(n_estimators=i)
    rfc.fit(X_train, y_train)
    pred_rfc = rfc.predict(X_test)
    print(accuracy_score(y_test, pred_rfc))

0.78391
0.7846
0.78404
0.78393
0.78389
0.78366
0.784265
0.784575
0.78401


In [30]:
for i in range(9,20,1):
    rfc = RandomForestClassifier(n_estimators=i)
    rfc.fit(X_train, y_train)
    pred_rfc = rfc.predict(X_test)
    print(accuracy_score(y_test, pred_rfc))

0.78354
0.78405
0.783875
0.783835
0.7839
0.783695
0.78426
0.783935
0.784305
0.78395
0.783905
